<a href="https://colab.research.google.com/github/Bibirara/Natural-Disaster-Shock/blob/main/Base%20Line%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade quantecon